# River Plate vs Libertad — Informe Pospartido (Ush Analytics)
**CONMEBOL Libertadores** — 14/08/2025 — Asunción (Ida)

> Datos simulados para portfolio; reemplazá `data/events.csv` con datos reales cuando los tengas.

In [ ]:
%pip install -q pandas numpy matplotlib mplsoccer jinja2

In [ ]:

from pathlib import Path
import pandas as pd, numpy as np, math
import matplotlib.pyplot as plt
from mplsoccer import Pitch
from jinja2 import Template

ROOT=Path.cwd(); DATA=ROOT/'data'; REPORT=ROOT/'report'; IMG=REPORT/'img'; PBI=ROOT/'powerbi_exports'; BRAND=ROOT/'brand'
for p in [REPORT, IMG, PBI]: p.mkdir(parents=True, exist_ok=True)

events=pd.read_csv(DATA/'events.csv'); matches=pd.read_csv(DATA/'matches.csv')
meta=matches.iloc[0].to_dict(); teams=[meta['home_team'], meta['away_team']]

for c in ['is_shot','is_goal','is_pass','is_def_action']:
    if c in events.columns: events[c]=pd.to_numeric(events[c], errors='coerce').fillna(0).astype(int)
for c in ['x','y','xg']:
    if c in events.columns: events[c]=pd.to_numeric(events[c], errors='coerce')

def xg_lite(row, xmax=120.0, ymax=80.0):
    dx=max(0.0, xmax-float(row.get('x',0))); dy=float(row.get('y',0))-ymax/2.0
    dist=(dx*dx+dy*dy)**0.5; ang=np.arctan2(abs(dy), max(1e-6, dx))
    val=1/(1+np.exp((dist-18)/4))*(0.6+0.4*(1-ang/1.57))
    return max(0.02, min(0.8, float(val)))

shots=events[events['is_shot']==1].copy()
if 'xg' not in shots.columns or shots['xg'].isna().all(): shots['xg']=shots.apply(xg_lite, axis=1)

kpis={}
for t in teams:
    sub=shots[shots['team']==t]
    kpis[t]={'shots':int(len(sub)),'goals':int(sub['is_goal'].sum()),'xg':float(round(sub['xg'].sum(),2))}

def ppda(df, team):
    if not set(['is_pass','is_def_action','y']).issubset(df.columns): return None
    ymax=max(100.0, float(df['y'].max()) or 100.0); yn=df['y']/ymax*100.0
    opp=((df['team']!=team)&(df['is_pass']==1)&(yn>=40)).sum()
    our=((df['team']==team)&(df['is_def_action']==1)&(yn>=40)).sum()
    return round(float(opp)/float(our),2) if our else None
ppda_vals={t:ppda(events,t) for t in teams}

# Shot map
pitch=Pitch(pitch_type='statsbomb', line_zorder=2); fig,ax=pitch.draw(figsize=(8,6))
for i,t in enumerate(teams):
    sub=shots[shots['team']==t]; s=(sub['xg'].fillna(0.06)*900).clip(lower=30)
    pitch.scatter(sub['x'],sub['y'],s=s,marker=['o','s'][i%2],alpha=0.7,label=t,ax=ax)
ax.legend(loc='lower left'); ax.set_title(f"Shot Map — {teams[0]} vs {teams[1]} ({meta['date']})")
out_shot=IMG/'shotmap.png'; plt.savefig(out_shot,dpi=200,bbox_inches='tight'); plt.show(); print('Guardado:', out_shot)

# xG race
shots['minute']=pd.to_numeric(shots['minute'], errors='coerce').fillna(0).astype(int)
shots['xg_val']=shots['xg'].fillna(0.06); mins=range(0, max(int(shots['minute'].max()),95)+1)
fig,ax=plt.subplots(figsize=(8,4.5))
for t in teams:
    tmp=shots[shots['team']==t].groupby('minute')['xg_val'].sum().reindex(mins, fill_value=0).cumsum()
    ax.plot(list(mins), tmp.values, label=t)
ax.set_xlabel('Minuto'); ax.set_ylabel('xG acumulado'); ax.legend()
out_xg=IMG/'xg_race.png'; plt.savefig(out_xg,dpi=200,bbox_inches='tight'); plt.show(); print('Guardado:', out_xg)

# Pass network (fallback si no hay receiver)
passes=events[(events['is_pass']==1) & (events['event_type']=='Pass')].dropna(subset=['player'])
team_focus=teams[1]; df=passes[passes['team']==team_focus].copy()
fig,ax=plt.subplots(figsize=(8,6)); pitch=Pitch(pitch_type='statsbomb', line_zorder=2); pitch.draw(ax=ax)
out_net=IMG/'pass_network.png'
if len(df)>0:
    locs=df.groupby('player')[['x','y']].mean()
    counts=df['player'].value_counts(); players=counts.index.tolist(); rows=[]
    for i,a in enumerate(players):
        for b in players[i+1:]:
            w=int((counts[a]+counts[b])/2); if w>=3: rows.append((a,b,w))
    for p,row in locs.iterrows():
        pitch.scatter([row['x']],[row['y']], s=300, alpha=0.85, ax=ax); ax.text(row['x'],row['y']-3,p,ha='center',va='top',fontsize=9)
    for a,b,w in rows:
        if a in locs.index and b in locs.index:
            xa,ya=locs.loc[a,['x','y']]; xb,yb=locs.loc[b,['x','y']]
            ax.plot([xa,xb],[ya,yb], linewidth=1+0.2*w, alpha=0.4)
    ax.set_title(f"Red de pases — {team_focus}")
plt.savefig(out_net,dpi=200,bbox_inches='tight'); plt.close(fig); print('Guardado:', out_net)

# Exports PBI
shots[['match_id','team','minute','x','y','is_goal','xg']].to_csv(PBI/'shots.csv', index=False)
import pandas as pd
pd.DataFrame([{'team':t, **kpis[t], 'ppda':ppda_vals[t]} for t in teams]).to_csv(PBI/'kpis.csv', index=False)

# Branded HTML with cover
from jinja2 import Template
HTML = "<!doctype html><html lang='es'><head><meta charset='utf-8'>\n<meta name='viewport' content='width=device-width,initial-scale=1'><title>{{ title }}</title>\n<style>\n:root{--navy:#0A2540;--cyan:#00C2FF;--blue:#5B86E5;--paper:#FFFFFF;--fog:#E6EEF6;--ink:#0A2540;--goal:#FF3366}\n*{box-sizing:border-box} body{font-family:Inter,system-ui,Segoe UI,Roboto,Arial,sans-serif;margin:0;background:#F6FAFF;color:var(--ink)}\n.wrap{max-width:980px;margin:0 auto;padding:16px}\n.card{background:var(--paper);border-radius:16px;box-shadow:0 6px 24px rgba(10,37,64,.08);padding:20px;margin:16px 0}\nh1{margin:0;font-size:32px} h2{margin-top:0}\n.kpis{display:grid;grid-template-columns:repeat(auto-fit,minmax(220px,1fr));gap:12px}\n.kpi{background:#EEF4FF;border-radius:12px;padding:12px}\n.kpi b{font-size:16px}\n.grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(260px,1fr));gap:16px}\nimg{max-width:100%;border-radius:12px}\n.badge{display:inline-flex;align-items:center;gap:12px;background:var(--navy);color:#fff;border-radius:999px;padding:6px 12px}\n.badge img{height:28px}\nfooter{font-size:12px;color:#567;text-align:center;padding:24px}\n.cover{background:linear-gradient(135deg,var(--navy),#081a30);color:#fff;padding:48px 0}\n.cover .title{display:flex;align-items:center;gap:16px}\n.cover h1{font-size:40px}\n.tag{display:inline-block;background:var(--cyan);color:#001018;padding:6px 10px;border-radius:999px;font-weight:600;margin-left:8px}\n.meta{margin-top:12px;color:#cfe3ff}\nhr.sep{border:none;border-top:2px solid var(--fog);margin:8px 0}\n@media print{@page{size:A4;margin:12mm} .pagebreak{page-break-before:always} a[href]:after{content:''}}\n</style></head>\n<body>\n  <!-- COVER -->\n  <section class=\"cover\">\n    <div class=\"wrap\">\n      <div class=\"title\">\n        <img src=\"{{ logo }}\" alt=\"Ush Analytics\" style=\"height:72px\">\n        <div>\n          <h1>Informe pospartido <span class=\"tag\">Ush Analytics</span></h1>\n          <div>{{ comp }} \u2014 {{ date }} \u2014 {{ venue }}</div>\n        </div>\n      </div>\n      <div class=\"meta\">\n        <div><b>Partido:</b> {{ away }} @ {{ home }} \u2014 Ida</div>\n        <div><b>Marcador:</b> {{ away_goals }}\u2013{{ home_goals }}</div>\n      </div>\n    </div>\n  </section>\n\n  <div class=\"wrap pagebreak\">\n    <!-- KPIs -->\n    <section class=\"card\">\n      <h2>KPIs</h2>\n      <div class=\"kpis\">\n        {% for t in teams %}\n        <div class=\"kpi\">\n          <div><b>{{ t }}</b></div>\n          <div>Tiros: {{ kpis[t].shots }}</div>\n          <div>Goles: {{ kpis[t].goals }}</div>\n          <div>xG: {{ \"%.2f\"|format(kpis[t].xg) }}</div>\n          <div>PPDA: {{ ppda[t] if ppda[t] is not none else \"\u2014\" }}</div>\n        </div>\n        {% endfor %}\n      </div>\n      <small>PPDA: pases rivales permitidos por acciones defensivas propias en zona alta. Valores bajos \u21d2 m\u00e1s presi\u00f3n.</small>\n    </section>\n\n    <!-- Visuales -->\n    <section class=\"card\">\n      <h2>Visuales</h2>\n      <div class=\"grid\">\n        <div><h3>Shot Map</h3><img src=\"{{ shotmap }}\" alt=\"Shot Map\"></div>\n        <div><h3>xG acumulado</h3><img src=\"{{ xgrace }}\" alt=\"xG acumulado\"></div>\n        <div style=\"grid-column:1/-1\">\n          <h3>Red de pases \u2014 {{ team_focus }}</h3>\n          <img src=\"{{ passnet }}\" alt=\"Red de pases\">\n        </div>\n      </div>\n    </section>\n\n    <!-- Comentarios -->\n    <section class=\"card\">\n      <h2>Notas r\u00e1pidas</h2>\n      <ul>\n        <li><b>Momentum:</b> River mejor\u00f3 su xG en el ST y marc\u00f3 en momentos clave <span style=\"color:var(--goal)\">({{ goals_away }})</span>.</li>\n        <li><b>Construcci\u00f3n:</b> Libertad prioriz\u00f3 mitad de cancha; pressing selectivo de River (PPDA m\u00e1s bajo).</li>\n        <li><b>Circulaci\u00f3n:</b> Foco en carril derecho para {{ team_focus }} con conexiones fuertes interior\u2013extremo.</li>\n      </ul>\n    </section>\n  </div>\n\n  <footer>\u00a9 Ush Analytics \u2014 Generado autom\u00e1ticamente.</footer>\n</body></html>\n"
title=f"{teams[1]} vs {teams[0]} — Ida {meta['competition']} ({meta['date']}, {meta['venue_city']})"
html=Template(HTML).render(
    title=title, logo=str(BRAND/'ush_logo_dark.svg'), comp=meta['competition'], date=meta['date'],
    venue=meta['venue_city'], away=teams[1], home=teams[0], away_goals=meta['away_goals'], home_goals=meta['home_goals'],
    teams=teams, kpis=kpis, ppda=ppda_vals, shotmap=str(out_shot), xgrace=str(out_xg), passnet=str(out_net),
    team_focus=teams[1], goals_away=meta['away_goals']
)
(REPORT/'river_libertad_report.html').write_text(html, encoding='utf-8'); print('Reporte:', REPORT/'river_libertad_report.html')
